In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from Finite_volume_method_1D import *

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Computer Modern",
    "figure.dpi": 200,
})

In [ ]:
chippath = '../../DataKian/Al_LT165chip7/'
Al_KID3_79_120 = KID_data(chippath,
                          lambda_ph_in_nm=673,
                          KIDno=2,
                          readout_power=89,
                          temp=120,
                          width=10,
                          length=310,
                          T_eff=0.225,
                          N0=17200000000,
                          sigma_IC=5,
                          D=15000)

chippath = '../../DataKian/bTa_LT278W2chip5/'
bTa_KID3_96_100 = KID_data(chippath,
                          lambda_ph_in_nm=673,
                          KIDno=3,
                          readout_power=96,
                          temp=100,
                          width=10,
                          length=90,
                          T_eff=0.100,
                          N0=30300000000,
                          sigma_IC=5,
                          D=50)

In [ ]:
bTa_KID3_96_100.fit_tail(start=250)
Al_KID3_79_120.fit_tail(start=510)

In [ ]:
# define simulation geometry
dt=0.1
dx=0.5
L=length # make sure dx cleanly divides L
N=int(L/dx)
x_borders=np.linspace(-L/2,L/2,N,endpoint=True)
x_centers=np.linspace(-L/2+dx,L/2-dx,N-1,endpoint=True)
D=np.ones_like(x_borders)*D_all

# IC
Q_init = np.exp(-0.5*(x_centers/sigma_init)**2)*N_init/(sigma_init*np.sqrt(2*np.pi))

print(N_init)
print(np.trapz(Q_init,x_centers))

# Simulation
T=len(phase)/4
times, Q_list_Al = simulate(Q_init,int(T/dt),dt,dx,D_all,R,Q0)

In [ ]:
Nqp=nqp_to_Nqp(Q_list_Al,x_centers)

thetas = Nqp_to_theta(Nqp,dthetadN,Q,f0,times)

plt.figure()
plt.semilogy(times,thetas,label='simulation')
plt.semilogy(t_range,phase,label='measurement')

# 'fit' of eta_pb
factor = np.max(phase)/np.max(thetas)
print('eta_pb = ', eta_pb*factor)

In [ ]:
# Theory: thomas' 1/t oplossing

def decay(A,R,Q0,t):
  return 2*Q0/((2*Q0/A+1)*np.exp(2*Q0*R*t)-1)

t_range = np.arange(len(phase))
Qs,Ts =np.meshgrid(Q_init,t_range)

theory = decay(Qs,R,Q0,Ts)


Nqp=nqp_to_Nqp(theory,x_centers)

thetas = Nqp_to_theta(Nqp,dthetadN,Q,f0,t_range)

plt.figure()
plt.semilogy(t_range,thetas,label='simulation')
plt.semilogy(t_range,phase,label='measurement')
plt.show()

In [ ]:
# Theory: thomas' 1/t oplossing

def decay(A,R,Q0,t):
  return 2*Q0/((2*Q0/A+1)*np.exp(2*Q0*R*t)-1)

t_range = np.arange(len(phase))

theory = decay(N_init/length,R,Q0,t_range)


Nqp=theory*length

thetas = Nqp_to_theta(Nqp,dthetadN,Q,f0,t_range)

plt.figure()
plt.semilogy(t_range,thetas,label='simulation')
plt.semilogy(t_range,phase,label='measurement')
plt.show()

In [ ]:
2*N0*np.sqrt(2*np.pi*k_B*T_eff*Delta)*np.exp(-Delta/(k_B*T_eff))

In [ ]:
tau